In [16]:
!git pull
# https://github.com/NirAndria/VALDOM_LLM_ENGINEERING.git


Already up to date.


In [17]:
import os 
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import ollama
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from smolagents import tool
import pandas as pd 
from smolagents import CodeAgent, LiteLLMModel


oc = ollama.Client("http://localhost:11435")
model = "deepseek-r1:32b"


In [18]:
doc_list = []
path_file = "VALDOM_LLM_ENGINEERING/data_RAG/processed_table/"
def generate_doc(path_file):
    for file_name in os.listdir(path_file):
        print(file_name)
        doc_list.append(TextLoader(path_file + file_name).load()[0])
        
generate_doc(path_file)

Age of MPs.txt
Age of MPs_Perc.txt
Age of newly elected MPs.txt
Education.txt
Ethnicity.txt
Ethnicity_Perc.txt
Female MPs.txt
Female MPs_Perc.txt
Men and Women.txt
New MPs.txt
New MPs_Perc.txt
New MPs_Tot.txt
Occupations 1997-2015.txt
Occupations 1997-2015_Perc.txt
Occupations 2017.txt
Length of Service.txt
Occupations 2019_.txt


In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n"],  # Prioritize keeping paragraphs together, then sentences, then words.
    chunk_size=500,  # Aim for chunks of around 256 characters
    chunk_overlap=16,  # Overlap chunks by 16 characters to preserve context
)

# Apply the splitter
doc_list = text_splitter.split_documents(doc_list)

In [20]:
from abc import abstractmethod, ABC
from langchain_core.documents import Document
from typing import List, Optional, Any

class RAGInterface(ABC):
    """
    Abstract class defining a generic RAG system. 
    
    This class ensures that all RAG implementations follow a common structure.
    """

    def __init__(self, name: str, knowledge_db: Optional[Any] = None):
        self.name = name  # Identifier for the RAG system
        self.knowledge_db = knowledge_db  # Storage backend (e.g., a vector database)

    @abstractmethod
    def retrieve(self, query: str) -> List[Document]:
        """
        Retrieve relevant contexts from the knowledge_db based on the query.

        Args:
            query (str): The user query.

        Returns:
            List[Document]: Retrieved document chunks.
        """
        pass

    @abstractmethod
    def generate(self, query: str, retrieved_contexts: List[Document]) -> str:
        """
        Generate a response based on the query and retrieved contexts.

        Args:
            query (str): The user query.
            retrieved_contexts (List[Document]): Relevant document chunks.

        Returns:
            str: The generated response.
        """
        pass


In [21]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(model="mistral")

In [22]:
query_list = []
question_path = "VALDOM_LLM_ENGINEERING/questions/"
for file in os.listdir(question_path):
    if ".txt" in file:
        with open(question_path + file,'r') as f:
            query_list.append(f.read())

qualitative_dataset =  pd.DataFrame({'user_input': query_list})
qualitative_dataset.head()

,user_input
0,"From 1997 to two years ago, how has the percen..."
1,What are the possible scenarios for forming a ...


In [23]:
from langchain_core.prompts import PromptTemplate
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
import pandas as pd

# 📝 Define a Structured Prompt for Generation
prompt_template_v0 = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {query} 
Context: {retrieved_contexts} 
Answer:
"""


# Create a prompt template from the string
gen_prompt_v0 = PromptTemplate.from_template(prompt_template_v0)

# 🔎 Define BM25-Based RAG System
def run_rag_on_df(rag: RAGInterface, eval_df: pd.DataFrame) -> pd.DataFrame:
    """
    Runs a RAG system on a dataset and store responses.
    
    Args:
        rag (RAGInterface): The RAG system to evaluate.
        eval_df (pd.DataFrame): DataFrame containing user queries and references.
    
    Returns:
        pd.DataFrame: The updated DataFrame including retrieved contexts and generated responses.
    """
    rag_df = eval_df.copy()

    # Retrieve relevant contexts for each query
    rag_df["retrieved_contexts"] = rag_df["user_input"].apply(rag.retrieve)

    # Generate answers using the retrieved contexts
    rag_df["response"] = rag_df.apply(
        lambda row: rag.generate(row["user_input"], row["retrieved_contexts"]), axis=1
    )

    # Convert retrieved documents to their text content
    rag_df["retrieved_contexts"] = rag_df["retrieved_contexts"].apply(
        lambda rcontexts: [rcontext.page_content for rcontext in rcontexts]
    )

    # Track the RAG implementation used
    rag_df["rag_name"] = rag.name

    return rag_df


# Retrival

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode

# Create a prompt template from the string
gen_prompt_v0 = PromptTemplate.from_template(prompt_template_v0)

# 🔎 Define BM25-Based RAG System
class BM25V0RAG(RAGInterface):
    """
    Sparse Retrieval RAG using BM25 and Qdrant.
    
    - Stores text chunks in Qdrant using BM25 sparse embeddings.
    - Retrieves the top-k relevant chunks based on keyword matching.
    - Uses a language model to generate answers from retrieved contexts.
    """

    def __init__(self, generation_model: OllamaLLM):
        # Initialize BM25 sparse embeddings
        sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25", cache_dir='.')

        # Store documents in Qdrant using sparse retrieval
        knowledge_db = QdrantVectorStore.from_documents(
            doc_list,  # Pre-processed text chunks
            embedding=OllamaEmbeddings(model="mistral"),  # Not used in this mode, but required
            sparse_embedding=sparse_embeddings,  # BM25 embeddings
            location=":memory:",  # Store in-memory (can be changed to persistent storage)
            collection_name="the_miser_sparse_v0",
            retrieval_mode=RetrievalMode.SPARSE,  # Use only sparse retrieval
        )

        # Define model name dynamically
        self.name = f"bm25_v0_{generation_model.model}"
        super().__init__(name=self.name, knowledge_db=knowledge_db)

        # Initialize the LLM and retriever
        self.llm = generation_model
        self.retriever = self.knowledge_db.as_retriever(
            search_type="similarity", search_kwargs={"k": 5}  # Retrieve top 5 matches
        )
        self.gen_prompt = gen_prompt_v0  # Use the structured prompt
    
    def retrieve(self, query: str) -> List[Document]:
        """
        Retrieves relevant documents using BM25 sparse retrieval.

        Args:
            query (str): The user query.

        Returns:
            List[Document]: The most relevant document chunks.
        """
        retrieved_contexts = self.retriever.invoke(query)
        return retrieved_contexts
    
    def generate(self, query: str, retrieved_contexts: List[Document]) -> str:
        """
        Generates a response using the retrieved contexts.

        Args:
            query (str): The user query.
            retrieved_contexts (List[Document]): Retrieved document chunks.

        Returns:
            str: The generated answer.
        """
        # Format retrieved contexts into a single string
        
        format_retrieved_contexts = "\n".join([rc.page_content for rc in retrieved_contexts])
        # Format the query with the retrieved contexts
        augmented_query = self.gen_prompt.format(
            name=self.name,
            query=query,
            retrieved_contexts=format_retrieved_contexts
        )

        # Generate the final response
        response = self.llm.invoke(augmented_query)
        return response

In [25]:
from abc import abstractmethod, ABC

# Initialize the generation model
generation_llm = OllamaLLM(model="deepseek-r1:32b")

# Initialize BM25 RAG system
bm25_v0 = BM25V0RAG(generation_model=generation_llm)
#bm25_v0_qualitative_df = run_rag_on_df(rag=bm25_v0, eval_df=qualitative_dataset)

In [26]:
rag = bm25_v0

# Agent

In [27]:
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool


In [28]:
@tool
def get_current_date() -> datetime.date:
    """
    This is a funciton to retrieve the current date. This tool takes no arguments.
    
    Args:

    Returns:
        A dictionary with today's date as a string in ISO format. It should look like this {'curren_date': AAAA.MM.DD}
    """
    print(f"Agent 'date_retriever' is executing the get_current_date tool.")
    try:
        current_date = datetime.now().date()  # Get the current date
        return current_date  # Return the date object directly
    except Exception as e:
        print(f"Error getting the current date: {str(e)}")
        return None 


In [29]:
@tool
def get_answer_from_rag(query: str) -> str:
    """Answer to a query given the documents given to a RAG
    
    Args:
        query: Query to be answered
        
    Returns:
        The answer of the query.
    """
    try:
        print(1)
        context = rag.retrieve(query)
    except Exception as e:
        print(f"Error : {str(e)}")
        return None
    
    try:
        print(2)
        print(f"query is: {query}")
        print(f"context is: {context}")
        answer = rag.generate(query, context)
        print(3)
    except Exception as e:
        print(f"Error : {str(e)}")
        return None 
    
    return answer 



In [30]:
eng_model = LiteLLMModel(
    model_id="ollama/deepseek-r1:32b", 
    api_base="http://localhost:11434/api/generate", 
    num_ctx=8192
)

date_agent = CodeAgent(
    tools=[get_current_date],#get_current_date,],
    model=eng_model,
    max_steps=10,
    name="date_agent",
    description="It is an agent who can get the current date.",
    additional_authorized_imports=["numpy", "datetime"],
)

RAG_agent = CodeAgent(
    tools=[get_answer_from_rag,],
    model=eng_model,
    max_steps=5,
    name="rag_agent",
    description="It is an agent who answer to any questions given the documents about the Parliament, the House of Commons passed in the RAG.",
)

/root/valdom_env/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [31]:
date_agent.prompt_templates['managed_agent'] = {'task': "You're a helpful agent named '{{name}}' and your main goal is to get the current date.\nYou have been submitted this task by your manager.\n---\nTask:\n{{task}}\n---\nFirst thing you have to do is to check if one of the tools that you have can solve partially or totally the task. Provide short answer.\n\nYour final_answer WILL HAVE to be in this format:\n YYYY-DD-MM \nIf your task resolution is not successful, please return as much context as possible, so that your manager can act upon this feedback.",
 'report': "Here is the final answer from your managed agent '{{name}}':\n{{final_answer}}"}
!git pull

# https://github.com/NirAndria/VALDOM_LLM_ENGINEERING.git

​

Already up to date.

​

import os 

from langchain_community.document_loaders import TextLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter

import ollama

from datetime import datetime

import requests

from bs4 import BeautifulSoup

from smolagents import tool

import pandas as pd 

from smolagents import CodeAgent, LiteLLMModel

​

​

oc = ollama.Client("http://localhost:11435")

model = "deepseek-r1:32b"

​

doc_list = []

path_file = "VALDOM_LLM_ENGINEERING/data_RAG/processed_table/"

def generate_doc(path_file):

    for file_name in os.listdir(path_file):

        print(file_name)

        doc_list.append(TextLoader(path_file + file_name).load()[0])

        

generate_doc(path_file)

Age of MPs.txt
Age of MPs_Perc.txt
Age of newly elected MPs.txt
Education.txt
Ethnicity.txt
Ethnicity_Perc.txt
Female MPs.txt
Female MPs_Perc.txt
Men and Women.txt
New MPs.txt
New MPs_Perc.txt
New MPs_Tot.txt
Occupations 1997-2015.txt
Occupations 1997-2015_Perc.txt
Occupations 2017.txt
Length of Service.txt
Occupations 2019_.txt

text_splitter = RecursiveCharacterTextSplitter(

    separators=["\n"],  # Prioritize keeping paragraphs together, then sentences, then words.

    chunk_size=500,  # Aim for chunks of around 256 characters

    chunk_overlap=16,  # Overlap chunks by 16 characters to preserve context

)

​

# Apply the splitter

doc_list = text_splitter.split_documents(doc_list)

​

from abc import abstractmethod, ABC

from langchain_core.documents import Document

from typing import List, Optional, Any

​

class RAGInterface(ABC):

    """

    Abstract class defining a generic RAG system. 

    

    This class ensures that all RAG implementations follow a common structure.

    """

​

    def __init__(self, name: str, knowledge_db: Optional[Any] = None):

        self.name = name  # Identifier for the RAG system

        self.knowledge_db = knowledge_db  # Storage backend (e.g., a vector database)

​

    @abstractmethod

    def retrieve(self, query: str) -> List[Document]:

        """

        Retrieve relevant contexts from the knowledge_db based on the query.

​

        Args:

            query (str): The user query.

​

        Returns:

            List[Document]: Retrieved document chunks.

        """

        pass

​

    @abstractmethod

    def generate(self, query: str, retrieved_contexts: List[Document]) -> str:

        """

        Generate a response based on the query and retrieved contexts.

​

        Args:

            query (str): The user query.

            retrieved_contexts (List[Document]): Relevant document chunks.

​

        Returns:

            str: The generated response.

        """

        pass

​

from langchain_ollama.llms import OllamaLLM

from langchain_ollama import OllamaEmbeddings

​

embedding = OllamaEmbeddings(model="mistral")

​

query_list = []

question_path = "VALDOM_LLM_ENGINEERING/questions/"

for file in os.listdir(question_path):

    if ".txt" in file:

        with open(question_path + file,'r') as f:

            query_list.append(f.read())

​

qualitative_dataset =  pd.DataFrame({'user_input': query_list})

qualitative_dataset.head()

	user_input
0 	From 1997 to two years ago, how has the percen...
1 	What are the possible scenarios for forming a ...

from langchain_core.prompts import PromptTemplate

from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode

import pandas as pd

​

# 📝 Define a Structured Prompt for Generation

prompt_template_v0 = """

You are an assistant for question-answering tasks.

Use the following pieces of retrieved context to answer the question.

If you don't know the answer, just say that you don't know.

Use three sentences maximum and keep the answer concise.

Question: {query} 

Context: {retrieved_contexts} 

Answer:

"""

​

​

# Create a prompt template from the string

gen_prompt_v0 = PromptTemplate.from_template(prompt_template_v0)

​

# 🔎 Define BM25-Based RAG System

def run_rag_on_df(rag: RAGInterface, eval_df: pd.DataFrame) -> pd.DataFrame:

    """

    Runs a RAG system on a dataset and store responses.

    

    Args:

        rag (RAGInterface): The RAG system to evaluate.

        eval_df (pd.DataFrame): DataFrame containing user queries and references.

    

    Returns:

        pd.DataFrame: The updated DataFrame including retrieved contexts and generated responses.

    """

    rag_df = eval_df.copy()

​

    # Retrieve relevant contexts for each query

    rag_df["retrieved_contexts"] = rag_df["user_input"].apply(rag.retrieve)

​

    # Generate answers using the retrieved contexts

    rag_df["response"] = rag_df.apply(

        lambda row: rag.generate(row["user_input"], row["retrieved_contexts"]), axis=1

    )

​

    # Convert retrieved documents to their text content

    rag_df["retrieved_contexts"] = rag_df["retrieved_contexts"].apply(

        lambda rcontexts: [rcontext.page_content for rcontext in rcontexts]

    )

​

    # Track the RAG implementation used

    rag_df["rag_name"] = rag.name

​

    return rag_df

​

Retrival

from langchain_core.prompts import PromptTemplate

from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode

​

# Create a prompt template from the string

gen_prompt_v0 = PromptTemplate.from_template(prompt_template_v0)

​

# 🔎 Define BM25-Based RAG System

class BM25V0RAG(RAGInterface):

    """

    Sparse Retrieval RAG using BM25 and Qdrant.

    

    - Stores text chunks in Qdrant using BM25 sparse embeddings.

    - Retrieves the top-k relevant chunks based on keyword matching.

    - Uses a language model to generate answers from retrieved contexts.

    """

​

    def __init__(self, generation_model: OllamaLLM):

        # Initialize BM25 sparse embeddings

        sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25", cache_dir='.')

​

        # Store documents in Qdrant using sparse retrieval

        knowledge_db = QdrantVectorStore.from_documents(

            doc_list,  # Pre-processed text chunks

            embedding=OllamaEmbeddings(model="mistral"),  # Not used in this mode, but required

            sparse_embedding=sparse_embeddings,  # BM25 embeddings

            location=":memory:",  # Store in-memory (can be changed to persistent storage)

            collection_name="the_miser_sparse_v0",

            retrieval_mode=RetrievalMode.SPARSE,  # Use only sparse retrieval

        )

​

        # Define model name dynamically

        self.name = f"bm25_v0_{generation_model.model}"

        super().__init__(name=self.name, knowledge_db=knowledge_db)

​

        # Initialize the LLM and retriever

        self.llm = generation_model

        self.retriever = self.knowledge_db.as_retriever(

            search_type="similarity", search_kwargs={"k": 5}  # Retrieve top 5 matches

        )

        self.gen_prompt = gen_prompt_v0  # Use the structured prompt

    

    def retrieve(self, query: str) -> List[Document]:

        """

        Retrieves relevant documents using BM25 sparse retrieval.

​

        Args:

            query (str): The user query.

​

        Returns:

            List[Document]: The most relevant document chunks.

        """

        retrieved_contexts = self.retriever.invoke(query)

        return retrieved_contexts

    

    def generate(self, query: str, retrieved_contexts: List[Document]) -> str:

        """

        Generates a response using the retrieved contexts.

​

        Args:

            query (str): The user query.

            retrieved_contexts (List[Document]): Retrieved document chunks.

​

        Returns:

            str: The generated answer.

        """

        # Format retrieved contexts into a single string

        

        format_retrieved_contexts = "\n".join([rc.page_content for rc in retrieved_contexts])

        # Format the query with the retrieved contexts

        augmented_query = self.gen_prompt.format(

            name=self.name,

            query=query,

            retrieved_contexts=format_retrieved_contexts

        )

​

        # Generate the final response

        response = self.llm.invoke(augmented_query)

        return response

from abc import abstractmethod, ABC

​

# Initialize the generation model

generation_llm = OllamaLLM(model="deepseek-r1:32b")

​

# Initialize BM25 RAG system

bm25_v0 = BM25V0RAG(generation_model=generation_llm)

#bm25_v0_qualitative_df = run_rag_on_df(rag=bm25_v0, eval_df=qualitative_dataset)

rag = bm25_v0

Agent

from smolagents import ToolCallingAgent, DuckDuckGoSearchTool

​

@tool

def get_current_date() -> datetime.date:

    """

    This is a funciton to retrieve the current date. This tool takes no arguments.

    

    Args:

​

    Returns:

        A dictionary with today's date as a string in ISO format. It should look like this {'curren_date': AAAA.MM.DD}

    """

    print(f"Agent 'date_retriever' is executing the get_current_date tool.")

    try:

        current_date = datetime.now().date()  # Get the current date

        return current_date  # Return the date object directly

    except Exception as e:

        print(f"Error getting the current date: {str(e)}")

        return None 

​

@tool

def get_answer_from_rag(query: str) -> str:

    """Answer to a query given the documents given to a RAG

    

    Args:

        query: Query to be answered

        

    Returns:

        The answer of the query.

    """

    try:

        print(1)

        context = rag.retrieve(query)

    except Exception as e:

        print(f"Error : {str(e)}")

        return None

    

    try:

        print(2)

        print(f"query is: {query}")

        print(f"context is: {context}")

        answer = rag.generate(query, context)

        print(3)

    except Exception as e:

        print(f"Error : {str(e)}")

        return None 

    

    return answer 

​

​

eng_model = LiteLLMModel(

    model_id="ollama/deepseek-r1:32b", 

    api_base="http://localhost:11434/api/generate", 

    num_ctx=8192

)

​

date_agent = CodeAgent(

    tools=[get_current_date],#get_current_date,],

    model=eng_model,

    max_steps=10,

    name="date_agent",

    description="It is an agent who can get the current date.",

    additional_authorized_imports=["numpy", "datetime"],

)

​

RAG_agent = CodeAgent(

    tools=[get_answer_from_rag,],

    model=eng_model,

    max_steps=5,

    name="rag_agent",

    description="It is an agent who answer to any questions given the documents about the Parliament, the House of Commons passed in the RAG.",

)

/root/valdom_env/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)

date_agent.prompt_templates['managed_agent'] = {'task': "You're a helpful agent named '{{name}}' and your main goal is to get the current date.\nYou have been submitted this task by your manager.\n---\nTask:\n{{task}}\n---\nFirst thing you have to do is to check if one of the tools that you have can solve partially or totally the task. Provide short answer.\n\nYour final_answer WILL HAVE to contain these parts:\n### 1. Task outcome (short version):\n### 2. Additional context (if relevant):\n\nPut all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be lost.\nAnd even if your task resolution is not successful, please return as much context as possible, so that your manager can act upon this feedback.",

 'report': "Here is the final answer from your managed agent '{{name}}':\n{{final_answer}}"}

​

RAG_agent.prompt_templates['managed_agent']['task'] = "You're a helpful agent named '{{name}}' and your main goal is to answer to a question using a RAG. The first thing you have to do is to use the tool named 'get_current_date', its purpose id to retrieve the information from the RAG. If you use an another method, you have to specify it. Don't use internet to get an answer, you have to use the RAG and nothing else.\nYou have been submitted this task by your manager.\n---\nTask:\n{{task}}\n---\nYou're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much information as possible to give them a clear understanding of the answer.\n\nYour final_answer WILL HAVE to contain these parts:\n### 1. Task outcome (short version):\n### 2. Task outcome (extremely detailed version):\n### 3. Additional context (if relevant):\n\nPut all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be lost.\nAnd even if your task resolution is not successful, please return as much context as possible, so that your manager can act upon this feedback."

manager_agent = CodeAgent(

    tools=[],

    model=eng_model,

    managed_agents=[date_agent,RAG_agent],

    additional_authorized_imports=["numpy", "pandas","datetime"], # these imports are useful for handling computations and table based data

)

qualitative_dataset.at[0, 'user_input'] = "what was the percentage of female members evolved across the House of Commons since 28 years ago ?"

agent_output = manager_agent.run(qualitative_dataset.at[0, 'user_input'])

print("Final output:")

print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what was the percentage of female members evolved across the House of Comm
RAG_agent.prompt_templates['managed_agent']['task'] = "You're a helpful agent named '{{name}}' and your main goal is to answer to a question using a RAG. The first thing you have to do is to use the tool named 'get_current_date', its purpose id to retrieve the information from the RAG. If you use an another method, you have to specify it. Don't use internet to get an answer, you have to use the RAG and nothing else.\nYou have been submitted this task by your manager.\n---\nTask:\n{{task}}\n---\nYou're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much information as possible to give them a clear understanding of the answer.\n\nYour final_answer WILL HAVE to contain these parts:\n### 1. Task outcome (short version):\n### 2. Task outcome (extremely detailed version):\n### 3. Additional context (if relevant):\n\nPut all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be lost.\nAnd even if your task resolution is not successful, please return as much context as possible, so that your manager can act upon this feedback."

In [32]:
manager_agent = CodeAgent(
    tools=[],
    model=eng_model,
    managed_agents=[date_agent,RAG_agent],
    additional_authorized_imports=["numpy", "pandas","datetime"], # these imports are useful for handling computations and table based data
)

In [35]:
qualitative_dataset.at[0, 'user_input'] = "what was the percentage of female members evolved across the House of Commons since 28 years ago ?"

In [36]:
agent_output = manager_agent.run(qualitative_dataset.at[0, 'user_input'])
print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what was the percentage of female members evolved across the House of Commons since 28 years ago ?              │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/deepseek-r1:32b ─────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Get current date to determine the relevant past year                                                           
  current_date = date_agent(task="get current date")                                                               
  print(f"Current date obtained: {current_date}")                                                                  
                                                                                                                   
  # Extract the current year and calculate 28 years ago                                                            
  from datetime import datetime                                                                                    
  date_obj = datetime.strptime(current_date, "%Y-%m-%d")                                                           
  current_year = date_obj.year                                                                                     
  past_year = current_year - 28                                                                                    
  print(f"Current year is {current_year}, and 28 years ago was {past_year}")                                       
                                                                                                                   
  # Query RAG for the current percentage of female members in House of Commons                                     
  current_female_percentage = rag_agent(task=f"What is the percentage of female members in the UK House of         
  Commons as of {current_year}?")                                                                                  
  print(f"Current percentage: {current_female_percentage}%")                                                       
                                                                                                                   
  # Query RAG for the percentage 28 years ago                                                                      
  past_female_percentage = rag_agent(task=f"What was the percentage of female members in the UK House of Commons   
  in {past_year}?")                                                                                                
  print(f"Percentage in {past_year}: {past_female_percentage}%")                                                   
                                                                                                                   
  # Calculate the evolution (change) in percentage                                                                 
  evolution = current_female_percentage - past_female_percentage                                                   
  final_answer(evolution)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭───────────────────────────────────────────── New run - date_agent ──────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'date_agent' and your main goal is to get the current date.                        │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ get current date                                                                                                │
│ ---                                                                                                             │
│ First thing you have to do is to check if one of the tools that you have can solve partially or totally the     │
│ task. You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give   │
│ as much information as possible to give them a clear understanding of the answer.                               │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/deepseek-r1:32b ─────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
Here is your code snippet:
<think>

</think>

To determine the percentage of women in the UK House of Commons in 1997, we can follow these steps:

1. **Identify Total MPs**: In the 1997 general election, there were a total of 659 Members of Parliament (MPs).

2. **Determine Number of Female MPs**: Historical records indicate that 120 women were elected to the House of 
Commons in 1997.

3. **Calculate the Percentage**:
   [
   \text{Percentage} = \left( \frac{\text{Number of Women}}{\text{Total MPs}} \right) \times 100
   \]
   Plugging in the numbers:
   [
   \text{Percentage} = \left( \frac{120}{659} \right) \times 100 \approx 18.2\%
   \]

### Final Answer:
The percentage of women in the UK House of Commons in 1997 was **18.2%**.
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_code>
Make sure to provide correct code blobs.

[Step 5: Duration 31.02 seconds| Input tokens: 30,034 | Output tokens: 3,182]

Reached max steps.

[Step 6: Duration 31.02 seconds| Input tokens: 38,226 | Output tokens: 3,796]

Final output:
<think>
Alright, I need to figure out how the percentage of female members in the UK House of Commons has changed over the past 28 years. First, I should determine what year is 28 years ago from now. If it's 2023, then 28 years prior would be 1995.

Next, I need to find out two key pieces of information: the percentage of female MPs in 1995 and the current percentage in 2023. To do this, I'll look up historical data from reliable sources like government records or reputable news outlets that track political demographics.

Once I have both percentages, I can calculate the evolution by subtracting the older percentage from the newer one. This will show whether there has been an increase, decrease, or no change in female representation over those 28 years.

Additionally, it would be helpful to provide some context about any significant events during this period that might have influenced changes in women's political participation, such as legislative reforms or cultural shif

In [ ]:
date_agent(task="get_current_year")